# Market Basket Analysis on UK Retailer Data

This analysis is focused on understanding the product association patterns seen in the transactions of an online retail store. The following steps are covered in this analysis:
- Data Cleaning
- Data Preparation
- Product association using Apriori algorithm for Frequent Itemset generation
- Product association using FP Growth algorithm for Frequent Itemset generation

## Dataset

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

## Importing libraries and data

In [13]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

In [14]:
# Reading the UK Retailer data from the UCI repository
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')

In [15]:
df.shape

(541909, 8)

In [16]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [18]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Data Cleaning

In [19]:
# Trimming leading and trailing spaces in Description column
df['Description'] = df['Description'].str.strip()
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1455
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [20]:
# Convert InvoiceNo and StockCode column to string
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df['StockCode'] = df['StockCode'].astype('str')
df.shape

(541909, 8)

In [21]:
# Remove rows corresponding to cancelled transactions
df = df[~df['InvoiceNo'].str.contains('C')]
df.shape

(532621, 8)

In [22]:
# Remove rows having negative value in Quantity column
df_pos = df[df['Quantity']>=0]
df_pos.shape

(531285, 8)

In [23]:
# Remove rows having missing values
df_mod = df_pos.dropna()
df_mod.shape

(397924, 8)

## Data Preparation

In [24]:
# Removing rows pertaining to irrelvant items 
exclude_items = ['BANK CHARGES', 'C2','DOT','M','PADS','POST']
df_mod = df_mod[~df_mod['StockCode'].isin(exclude_items)]
df_mod.shape

(396370, 8)

In [25]:
# Creating basket with horizontal data layout
basket = (df_mod.groupby(['InvoiceNo', 'StockCode'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))
basket.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Encoding basket data
def basket_encoder(x):
    if x>=1:
        return 1
    else:
        return 0

In [27]:
basket_encoded = basket.applymap(basket_encoder)

In [28]:
# Removing single item transactions
basket_encoded['UNIQ_ITEMS'] = basket_encoded.sum(axis = 1)
basket_encoded_filt = basket_encoded[basket_encoded['UNIQ_ITEMS']>=2].drop(columns = 'UNIQ_ITEMS')
basket_encoded_filt.shape

(17092, 3659)

In [29]:
# StockCode to Item Description mapping

# Removing duplicates and blank value mappings for the same stockcode
item_desc = df_mod[['StockCode','Description']].sort_values(['StockCode','Description'], ascending = False).reset_index().drop(columns = 'index')
item_desc = item_desc.drop_duplicates(subset=['StockCode'], keep = 'first').sort_values(['StockCode'], ascending = True).set_index('StockCode')
# 3659

# Creating stockcode : description item dictionary
item_dict = item_desc.to_dict()['Description']

In [30]:
# Changing headers from item code to item description
new_headers = []
for x in basket_encoded_filt.columns:
    new_headers.append(item_dict[x])
    
basket_encoded_filt.columns = new_headers

## Apriori Algorithm

In [31]:
# Getting frequent itemsets using Apriori algorithm
df_freq_itemsets_ap = apriori(basket_encoded_filt, min_support = 0.03, use_colnames=True).sort_values('support', ascending = False)

In [32]:
# Adding length of itemsets
df_freq_itemsets_ap['itemset_length'] = df_freq_itemsets_ap['itemsets'].apply(lambda x: len(x))

In [35]:
df_freq_itemsets_ap

,support,itemsets,itemset_length
107,0.114615,(WHITE HANGING HEART T-LIGHT HOLDER),1
45,0.098409,(REGENCY CAKESTAND 3 TIER),1
104,0.092675,(JUMBO BAG RED RETROSPOT),1
100,0.080330,(ASSORTED COLOUR BIRD ORNAMENT),1
92,0.079862,(PARTY BUNTING),1
...,...,...,...
110,0.030365,"(LUNCH BAG SUKI DESIGN, LUNCH BAG RED SPOTTY)",2
22,0.030248,(BLUE HARMONICA IN BOX),1
17,0.030248,(RED TOADSTOOL LED NIGHT LIGHT),1
109,0.030248,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED SPOTTY)",2


In [33]:
# Applying Association rules
df_asso_rules_ap = association_rules(df_freq_itemsets_ap, metric = 'lift').sort_values(['lift','confidence'], ascending = False)

In [34]:
df_asso_rules_ap

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.040428,0.045635,0.031652,0.782923,17.156058,0.029807,4.396440
3,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.045635,0.040428,0.031652,0.693590,17.156058,0.029807,3.131657
4,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.045869,0.051311,0.031009,0.676020,13.175075,0.028655,2.928238
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.051311,0.045869,0.031009,0.604333,13.175075,0.028655,2.411448
7,(LUNCH BAG PINK POLKADOT),(LUNCH BAG RED SPOTTY),0.054411,0.075357,0.030599,0.562366,7.462696,0.026499,2.112821
6,(LUNCH BAG RED SPOTTY),(LUNCH BAG PINK POLKADOT),0.075357,0.054411,0.030599,0.406056,7.462696,0.026499,1.592050
1,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.050842,0.092675,0.031945,0.628308,6.779702,0.027233,2.441070
0,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.092675,0.050842,0.031945,0.344697,6.779702,0.027233,1.448425
8,(LUNCH BAG SUKI DESIGN),(LUNCH BAG RED SPOTTY),0.061023,0.075357,0.030365,0.497603,6.603285,0.025767,1.840463
9,(LUNCH BAG RED SPOTTY),(LUNCH BAG SUKI DESIGN),0.075357,0.061023,0.030365,0.402950,6.603285,0.025767,1.572695


## Findings

<code><b>GREEN REGENCY TEACUP AND SAUCER</b></code> and <code><b>ROSES REGENCY TEACUP AND SAUCER</b></code> have the highest Lift value of 17.1561. This indicates that the association between them is very strong and positively correlated. Green -> Roses has higher confidence value of 0.78 indicating that 78% of the times when the Green cup and saucer set is purchased, the Roses set is also purchased.

## FP Growth Algorithm

In [36]:
# Getting frequent itemsets using FP Growth algorithm
df_freq_itemsets_fp = fpgrowth(basket_encoded_filt, min_support = 0.03, use_colnames=True).sort_values('support', ascending = False)

In [37]:
# Adding length of itemsets
df_freq_itemsets_fp['itemset_length'] = df_freq_itemsets_fp['itemsets'].apply(lambda x: len(x))

In [38]:
df_freq_itemsets_fp

,support,itemsets,itemset_length
0,0.114615,(WHITE HANGING HEART T-LIGHT HOLDER),1
68,0.098409,(REGENCY CAKESTAND 3 TIER),1
37,0.092675,(JUMBO BAG RED RETROSPOT),1
1,0.080330,(ASSORTED COLOUR BIRD ORNAMENT),1
85,0.079862,(PARTY BUNTING),1
...,...,...,...
113,0.030365,"(LUNCH BAG SUKI DESIGN, LUNCH BAG RED SPOTTY)",2
11,0.030248,(RED TOADSTOOL LED NIGHT LIGHT),1
60,0.030248,(BLUE HARMONICA IN BOX),1
112,0.030248,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED SPOTTY)",2


In [39]:
# Applying Association rules
df_asso_rules_fp = association_rules(df_freq_itemsets_fp, metric = 'lift').sort_values(['lift','confidence'], ascending = False)

In [40]:
df_asso_rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.040428,0.045635,0.031652,0.782923,17.156058,0.029807,4.396440
3,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.045635,0.040428,0.031652,0.693590,17.156058,0.029807,3.131657
4,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.045869,0.051311,0.031009,0.676020,13.175075,0.028655,2.928238
5,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.051311,0.045869,0.031009,0.604333,13.175075,0.028655,2.411448
7,(LUNCH BAG PINK POLKADOT),(LUNCH BAG RED SPOTTY),0.054411,0.075357,0.030599,0.562366,7.462696,0.026499,2.112821
6,(LUNCH BAG RED SPOTTY),(LUNCH BAG PINK POLKADOT),0.075357,0.054411,0.030599,0.406056,7.462696,0.026499,1.592050
1,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.050842,0.092675,0.031945,0.628308,6.779702,0.027233,2.441070
0,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.092675,0.050842,0.031945,0.344697,6.779702,0.027233,1.448425
8,(LUNCH BAG SUKI DESIGN),(LUNCH BAG RED SPOTTY),0.061023,0.075357,0.030365,0.497603,6.603285,0.025767,1.840463
9,(LUNCH BAG RED SPOTTY),(LUNCH BAG SUKI DESIGN),0.075357,0.061023,0.030365,0.402950,6.603285,0.025767,1.572695


## Findings

FP Growth algorithm generates the same output as Apriori for this analysis. The Frequent Itemset of both the algorithms match exactly.